In [3]:
import pandas as pd
import numpy as np

seds = pd.read_csv('seds.csv') #1
revenue_to_provider = pd.read_csv('revenue_to_provider.csv') #2
filter_pack_concentrations_weekly = pd.read_csv('filter_pack_concentration_weekly.csv') #3
energy_workforce_training = pd.read_csv('energy_workforce_training.csv') #4
vehicles = pd.read_csv('vehicles.csv') #5
land_temp_by_city = pd.read_csv('land_temp_by_city.csv') #6
land_temp_by_state = pd.read_csv('land_temp_by_state.csv') #7
power_plants = pd.read_csv('power_plants.csv') #8

In [15]:
power_plants.head()

,longitude,latitude,plant_name,utility_name,sector_name,city,county,state_name,primary_source,total_power,source_description,tech_description
0,-87.356823,33.458665,Bankhead Dam,Alabama Power Co,Electric Utility,Northport,as,Alabama,hydroelectric,56.0,g,Conventional Hydroelectric
1,-88.010300,31.006900,Barry,Alabama Power Co,Electric Utility,Bucks,Mobile,Alabama,coal,2160.5,"Coal = 1088.5 MW, Natural Gas = 1072 MW",Conventional Steam Coal; Natural Gas Fired Com...
2,-86.283056,32.583889,Walter Bouldin Dam,Alabama Power Co,Electric Utility,Wetumpka,Elmore,Alabama,hydroelectric,225.9,Hydroelectric = 225.9 MW,Conventional Hydroelectric
3,-85.970800,34.012800,Gadsden,Alabama Power Co,Electric Utility,Gadsden,Etowah,Alabama,natural gas,130.0,Natural Gas = 130 MW,Natural Gas Steam Turbine
4,-87.196486,33.644344,Gorgas,Alabama Power Co,Electric Utility,Parrish,Walker,Alabama,coal,1034.0,Coal = 1034 MW,Conventional Steam Coal


In [16]:
power_plants.describe()

,longitude,latitude,total_power
count,7938.000000,7938.000000,7826.000000
mean,-95.245314,39.134308,135.639215
std,18.669527,5.741334,357.091293
min,-171.712439,-17.287022,0.100000
25%,-112.859600,35.218360,3.025000
50%,-91.811438,39.545654,11.500000
75%,-79.305587,42.554413,81.000000
max,-38.914783,71.292000,7079.000000
